## Capstone Project - Battle of Neighbour hood
### Author : Umeshpratap Singh
#### This project is final journey towards IBM -Data Science Certification
It is a capstone proejct to demonstrate my learing in various data science subjects using python learned over course of IBM data science certification


In [3]:
import pandas as pd
import numpy as np

In [4]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!
